In [1]:
from pyspark.sql import SparkSession
import os
import pandas as pd
import time
import string
import pathlib
import random
import threading
import time
from urllib.parse import urlsplit, urlunsplit
import requests
import json
from py4j.protocol import Py4JJavaError, Py4JError
import glob
import psutil

In [2]:
# Global configuration
SPARK_MEMORY = 900
SPARK_CORES = 60
DBHOST = 'postgres'
QUERY_TIMEOUT = 60 * 30

In [3]:
def create_spark():
    spark = SparkSession.builder \
        .appName("app") \
        .master(f'local[{SPARK_CORES}]') \
        .config("spark.driver.memory", f'{SPARK_MEMORY}g') \
        .config("spark.executor.memory", f'{SPARK_MEMORY}g') \
        .config("spark.memory.offHeap.enabled",False) \
        .config("spark.jars", "postgresql-42.3.3.jar") \
        .getOrCreate()
    return spark

In [4]:
def extract_metrics(spark, group_id):
    parsed = list(urlsplit(spark.sparkContext.uiWebUrl))
    host_port = parsed[1]
    parsed[1] = 'localhost' + host_port[host_port.find(':'):]
    API_URL = f'{urlunsplit(parsed)}/api/v1'

    app_id = spark.sparkContext.applicationId
    sql_queries = requests.get(API_URL + f'/applications/{app_id}/sql', params={'length': '100000'}).json()
    query_ids = [q['id'] for q in sql_queries if q['description'] == group_id]
    if (len(query_ids) == 0):
        print(f'query with group {group_id} not found')
        return None
    query_id = query_ids[0]
    print(f'query id: {query_id}')
    
    query_details = requests.get(API_URL + f'/applications/{app_id}/sql/{query_id}',
                                 params={'details': 'true', 'planDescription': 'true'}).json()
    
    success_job_ids = query_details['successJobIds']
    running_job_ids = query_details['runningJobIds']
    failed_job_ids = query_details['failedJobIds']
    
    job_ids = success_job_ids + running_job_ids + failed_job_ids
    
    job_details = [requests.get(API_URL + f'/applications/{app_id}/jobs/{jid}').json() for jid in job_ids]
    
    job_stages = {}
    
    for j in job_details:
        stage_ids = j['stageIds']
        
        stage_params = {'details': 'true', 'withSummaries': 'true'}
        stages = [requests.get(API_URL + f'/applications/{app_id}/stages/{sid}', stage_params) for sid in stage_ids]
        
        job_stages[j['jobId']] = [stage.json() for stage in stages if stage.status_code == 200] # can be 404
    
    return query_details, job_details, job_stages

In [5]:
def import_db(spark, dbname):
    
    username = dbname
    password = dbname
    dbname = dbname

    df_tables = spark.read.format("jdbc") \
    .option("url", f'jdbc:postgresql://{DBHOST}:5432/{dbname}') \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "information_schema.tables") \
    .option("user", username) \
    .option("password", password) \
    .load()

    for idx, row in df_tables.toPandas().iterrows():
        if row.table_schema == 'public':
            table_name = row.table_name
            df = spark.read.format("jdbc") \
                .option("url", f'jdbc:postgresql://{DBHOST}:5432/{dbname}') \
                .option("driver", "org.postgresql.Driver") \
                .option("dbtable", table_name) \
                .option("user", username) \
                .option("password", password) \
                .load()
    
            print(table_name)
            #print(df.show())
            df.createOrReplaceTempView(table_name)

def random_str(size=16, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

def set_group_id(spark):
    group_id = random_str()
    spark.sparkContext.setJobGroup(group_id, group_id)
    return group_id

def cancel_query(spark, seconds, group_id):
    time.sleep(seconds)
    print("cancelling jobs with id " + group_id)
    print(spark.sparkContext.cancelJobGroup(group_id))
    print("cancelled job")

def cancel_query_after(spark, seconds):
    group_id = random_str()
    spark.sparkContext.setJobGroup(group_id, group_id)
    threading.Thread(target=cancel_query, args=(spark, seconds, group_id,)).start()
    return group_id
    
def run_query(spark, file):
    with open(file, 'r') as f:
        query = '\n'.join(filter(lambda line: not line.startswith('limit') and not line.startswith('-'), f.readlines()))
        
        print("running query: \n" + query)
        return spark.sql(query)

def get_resource_usage(t):
    return {
        'time': t,
        'memory': psutil.virtual_memory(),
        'cpu': psutil.cpu_percent(interval=None, percpu=True),
        'cpu_total': psutil.cpu_percent(interval=None, percpu=False)
    }

In [6]:
resource_usage = []

def measure_resource_usage(resource_usage):
    t = threading.current_thread()
    secs = 0
    while getattr(t, "do_run", True):
        resource_usage.append(get_resource_usage(secs))
        #print("resource usage: " + str(resource_usage))
        secs += 1
        time.sleep(1)

def benchmark_query(spark, query, respath, run):
    spark.sparkContext._jvm.System.gc()
    start_time = time.time()

    resource_usage = []

    measure_thread = threading.Thread(target=measure_resource_usage, args=(resource_usage, ))
    measure_thread.start()

    group_id = cancel_query_after(spark, QUERY_TIMEOUT)
    df1 = run_query(spark, query)
    df1.show()

    measure_thread.do_run = False

    end_time = time.time()
    diff_time = end_time - start_time

    execution, jobs, job_stages = extract_metrics(spark, group_id)

    with open(respath + f'/resource-usage-{run}.json', 'w') as f:
        f.write(json.dumps(resource_usage, indent=2))

    resource_list = map(lambda r: [r['time'], r['memory'].used, r['cpu_total']], resource_usage)
    resource_df = pd.DataFrame(resource_list, columns = ['time', 'memory_used', 'cpu_used'])
    resource_df.to_csv(respath + f'/resource-usage-{run}.csv')

    peak_memory = max(map(lambda r: r['memory'].used, resource_usage)) / (1000 * 1000 * 1000) # GB

    if execution is not None:
            with open(respath + f'/execution-{run}.json', 'w') as f:
                f.write(json.dumps(execution, indent=2))
            with open(respath + f'/jobs-{run}.json', 'w') as f:
                f.write(json.dumps(jobs, indent=2))
            with open(respath + f'/stages-{run}.json', 'w') as f:
                f.write(json.dumps(job_stages, indent=2))
    return (diff_time, peak_memory)

def benchmark(spark, dbname, query_file, mode, run):
    #spark.sql("SET spark.sql.yannakakis.enabled = false").show()
    # run the query once to warm up Spark (load the relation in memory)
    #df0 = run_query(query)
    #df0.show()
    
    query_name = os.path.basename(query_file)

    respath = f'benchmark-results-{dbname}/' + query_name + "/" + mode
    pathlib.Path(respath).mkdir(parents=True, exist_ok=True)

    if mode == "opt":
        spark.sql("SET spark.sql.yannakakis.enabled = true").show()
    elif mode == "ref":
        spark.sql("SET spark.sql.yannakakis.enabled = false").show()
    else:
        return []

    try:
        (runtime, peak_memory) = benchmark_query(spark, query_file, respath, run)
        return [query_name, runtime, peak_memory, mode, run]
    except Py4JError as e:
        print('timeout or error: ' + str(e))
        return [query_name, None, None, mode, run]

def benchmark_all(dbname, mode, runs, queries):
    spark = create_spark()
    import_db(spark, dbname)

    results_df = df = pd.DataFrame([], columns = ['query', 'runtime', 'peak_memory', 'mode', 'run'])
    results_file = f'benchmark-results-{dbname}/results-{mode}.csv'
    if (os.path.exists(results_file)):
        results_df = pd.read_csv(results_file)
        print(results_file)

    for run in runs:
        for q in queries:
            results = [benchmark(spark, dbname, q, mode, run)]
            new_df = pd.DataFrame(results, columns = ['query', 'runtime', 'peak_memory', 'mode', 'run'])
            results_df = pd.concat([results_df, new_df], ignore_index=True)
            results_df.to_csv(f'benchmark-results-{dbname}/results-{mode}.csv')
            print(results_df)
    

## SNAP Benchmark

In [9]:
#### benchmark configuration
group_in_leaves = False
dbname = 'snap'
tablename = 'patents'
mode = 'opt'
runs = ['01', '02', '03', '04', '05', '06']
####

queries = sorted(glob.glob(f'snap-queries/{tablename}/*'))
#queries = sorted(glob.glob(f'snap-queries/{tablename}/tree*'))
#queries = ['snap-queries/patents/tree01.sql']

print('running queries: ' + str(queries))
benchmark_all(dbname, mode, runs, queries)

running queries: ['snap-queries/patents/path02.sql', 'snap-queries/patents/path03.sql', 'snap-queries/patents/path04.sql', 'snap-queries/patents/path05.sql', 'snap-queries/patents/path06.sql', 'snap-queries/patents/path07.sql', 'snap-queries/patents/path08.sql', 'snap-queries/patents/tree01.sql', 'snap-queries/patents/tree02.sql', 'snap-queries/patents/tree03.sql']
patents
wiki
google
dblp
benchmark-results-snap/results-opt.csv
+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.yannaka...| true|
+--------------------+-----+

running query: 
select count(*) from patents p1, patents p2, patents p3 where p1.toNode = p2.fromNode AND p2.toNode = p3.fromNode



23/11/16 20:04:33 WARN RewriteJoinsAsSemijoins: applying yannakakis rewriting to join: Aggregate [toprettystring(count(1), Some(GMT)) AS toprettystring(count(1))#6930]
+- Project
   +- Join Inner, (toNode#6923 = fromNode#6924)
      :- Project [tonode#6923]
      :  +- Join Inner, (toNode#6888 = fromNode#6922)
      :     :- Project [tonode#6888]
      :     :  +- Filter isnotnull(toNode#6888)
      :     :     +- Relation [fromnode#6887,tonode#6888] JDBCRelation(patents) [numPartitions=1]
      :     +- Filter (isnotnull(fromNode#6922) AND isnotnull(toNode#6923))
      :        +- Relation [fromnode#6922,tonode#6923] JDBCRelation(patents) [numPartitions=1]
      +- Project [fromnode#6924]
         +- Filter isnotnull(fromNode#6924)
            +- Relation [fromnode#6924,tonode#6925] JDBCRelation(patents) [numPartitions=1]

23/11/16 20:04:33 WARN RewriteJoinsAsSemijoins: agg(project(join))
23/11/16 20:04:33 WARN RewriteJoinsAsSemijoins: items: List(Project [tonode#6888]
+- Filter isnot

+---------+
| count(1)|
+---------+
|361611968|
+---------+

query id: 192
    Unnamed: 0       query    runtime  peak_memory mode run
0          0.0  path02.sql  16.945872    25.633620  opt   1
1          1.0  path03.sql  15.443754    37.772816  opt   1
2          2.0  path04.sql  16.042656    53.596385  opt   1
3          3.0  path05.sql  19.794976    71.539524  opt   1
4          4.0  path06.sql  19.110140    81.005621  opt   1
..         ...         ...        ...          ...  ...  ..
56        56.0  path08.sql  20.148151   386.348278  opt   6
57        57.0  tree01.sql  12.611429   387.085025  opt   6
58        58.0  tree02.sql  15.623933   387.631641  opt   6
59        59.0  tree03.sql  15.367422   388.450263  opt   6
60         NaN  path02.sql  15.108901    30.406738  opt  01

[61 rows x 6 columns]
+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.yannaka...| true|
+--------------------+-----+

running query: 
select count(*) from

23/11/16 20:04:49 WARN RewriteJoinsAsSemijoins: applying yannakakis rewriting to join: Aggregate [toprettystring(count(1), Some(GMT)) AS toprettystring(count(1))#6989]
+- Project
   +- Join Inner, (toNode#6982 = fromNode#6983)
      :- Project [tonode#6982]
      :  +- Join Inner, (toNode#6980 = fromNode#6981)
      :     :- Project [tonode#6980]
      :     :  +- Join Inner, (toNode#6888 = fromNode#6979)
      :     :     :- Project [tonode#6888]
      :     :     :  +- Filter isnotnull(toNode#6888)
      :     :     :     +- Relation [fromnode#6887,tonode#6888] JDBCRelation(patents) [numPartitions=1]
      :     :     +- Filter (isnotnull(fromNode#6979) AND isnotnull(toNode#6980))
      :     :        +- Relation [fromnode#6979,tonode#6980] JDBCRelation(patents) [numPartitions=1]
      :     +- Filter (isnotnull(fromNode#6981) AND isnotnull(toNode#6982))
      :        +- Relation [fromnode#6981,tonode#6982] JDBCRelation(patents) [numPartitions=1]
      +- Project [fromnode#6983]
   

+----------+
|  count(1)|
+----------+
|1380939105|
+----------+

query id: 195
    Unnamed: 0       query    runtime  peak_memory mode run
0          0.0  path02.sql  16.945872    25.633620  opt   1
1          1.0  path03.sql  15.443754    37.772816  opt   1
2          2.0  path04.sql  16.042656    53.596385  opt   1
3          3.0  path05.sql  19.794976    71.539524  opt   1
4          4.0  path06.sql  19.110140    81.005621  opt   1
..         ...         ...        ...          ...  ...  ..
57        57.0  tree01.sql  12.611429   387.085025  opt   6
58        58.0  tree02.sql  15.623933   387.631641  opt   6
59        59.0  tree03.sql  15.367422   388.450263  opt   6
60         NaN  path02.sql  15.108901    30.406738  opt  01
61         NaN  path03.sql  16.123209    39.175430  opt  01

[62 rows x 6 columns]
+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.yannaka...| true|
+--------------------+-----+

running query: 
select count(*)

...

+-----------+
|   count(1)|
+-----------+
|15961425879|
+-----------+

query id: 363
     Unnamed: 0       query    runtime  peak_memory mode run
0           0.0  path02.sql  16.945872    25.633620  opt   1
1           1.0  path03.sql  15.443754    37.772816  opt   1
2           2.0  path04.sql  16.042656    53.596385  opt   1
3           3.0  path05.sql  19.794976    71.539524  opt   1
4           4.0  path06.sql  19.110140    81.005621  opt   1
..          ...         ...        ...          ...  ...  ..
113         NaN  path05.sql  19.714842    43.201757  opt  06
114         NaN  path06.sql  19.386428    48.873386  opt  06
115         NaN  path07.sql  20.049885    50.018308  opt  06
116         NaN  path08.sql  21.900296    49.182491  opt  06
117         NaN  tree01.sql  16.427852    46.532780  opt  06

[118 rows x 6 columns]
+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.yannaka...| true|
+--------------------+-----+

running query

23/11/16 20:23:32 WARN RewriteJoinsAsSemijoins: applying yannakakis rewriting to join: Aggregate [toprettystring(count(1), Some(GMT)) AS toprettystring(count(1))#13515]
+- Project
   +- Join Inner, (toNode#13504 = fromNode#13509)
      :- Project [tonode#13504]
      :  +- Join Inner, (toNode#13504 = fromNode#13507)
      :     :- Project [tonode#13504]
      :     :  +- Join Inner, (toNode#13502 = fromNode#13505)
      :     :     :- Project [tonode#13502, tonode#13504]
      :     :     :  +- Join Inner, (toNode#13502 = fromNode#13503)
      :     :     :     :- Project [tonode#13502]
      :     :     :     :  +- Join Inner, (toNode#6888 = fromNode#13501)
      :     :     :     :     :- Project [tonode#6888]
      :     :     :     :     :  +- Filter isnotnull(toNode#6888)
      :     :     :     :     :     +- Relation [fromnode#6887,tonode#6888] JDBCRelation(patents) [numPartitions=1]
      :     :     :     :     +- Filter (isnotnull(fromNode#13501) AND isnotnull(toNode#13502))


+------------+
|    count(1)|
+------------+
|271470641431|
+------------+

query id: 366
     Unnamed: 0       query    runtime  peak_memory mode run
0           0.0  path02.sql  16.945872    25.633620  opt   1
1           1.0  path03.sql  15.443754    37.772816  opt   1
2           2.0  path04.sql  16.042656    53.596385  opt   1
3           3.0  path05.sql  19.794976    71.539524  opt   1
4           4.0  path06.sql  19.110140    81.005621  opt   1
..          ...         ...        ...          ...  ...  ..
114         NaN  path06.sql  19.386428    48.873386  opt  06
115         NaN  path07.sql  20.049885    50.018308  opt  06
116         NaN  path08.sql  21.900296    49.182491  opt  06
117         NaN  tree01.sql  16.427852    46.532780  opt  06
118         NaN  tree02.sql  18.428829    48.369529  opt  06

[119 rows x 6 columns]
+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.yannaka...| true|
+--------------------+-----+

running 

23/11/16 20:23:51 WARN RewriteJoinsAsSemijoins: applying yannakakis rewriting to join: Aggregate [toprettystring(count(1), Some(GMT)) AS toprettystring(count(1))#13625]
+- Project
   +- Join Inner, (toNode#13616 = fromNode#13619)
      :- Project [tonode#13616]
      :  +- Join Inner, (toNode#13614 = fromNode#13617)
      :     :- Project [tonode#13614, tonode#13616]
      :     :  +- Join Inner, (toNode#13610 = fromNode#13615)
      :     :     :- Project [tonode#13610, tonode#13614]
      :     :     :  +- Join Inner, (toNode#13610 = fromNode#13613)
      :     :     :     :- Project [tonode#13610]
      :     :     :     :  +- Join Inner, (toNode#13608 = fromNode#13611)
      :     :     :     :     :- Project [tonode#13608, tonode#13610]
      :     :     :     :     :  +- Join Inner, (toNode#13608 = fromNode#13609)
      :     :     :     :     :     :- Project [tonode#13608]
      :     :     :     :     :     :  +- Join Inner, (toNode#6888 = fromNode#13607)
      :     :     :  

+-------------+
|     count(1)|
+-------------+
|6483182708709|
+-------------+

query id: 369
     Unnamed: 0       query    runtime  peak_memory mode run
0           0.0  path02.sql  16.945872    25.633620  opt   1
1           1.0  path03.sql  15.443754    37.772816  opt   1
2           2.0  path04.sql  16.042656    53.596385  opt   1
3           3.0  path05.sql  19.794976    71.539524  opt   1
4           4.0  path06.sql  19.110140    81.005621  opt   1
..          ...         ...        ...          ...  ...  ..
115         NaN  path07.sql  20.049885    50.018308  opt  06
116         NaN  path08.sql  21.900296    49.182491  opt  06
117         NaN  tree01.sql  16.427852    46.532780  opt  06
118         NaN  tree02.sql  18.428829    48.369529  opt  06
119         NaN  tree03.sql  17.029732    49.062527  opt  06

[120 rows x 6 columns]
cancelling jobs with id M4KAC0OLE8QOH7FQ
None
cancelled job
cancelling jobs with id LHC7B1OYNI154H35
None
cancelled job
cancelling jobs with id EEMTB

## LSQB Benchmark

In [9]:
#### benchmark configuration
group_in_leaves = False
dbname = 'lsqb'
#mode = 'opt'
runs = ['01', '02', '03']
runs = ['05']
####

#queries = sorted(glob.glob(f'snap-queries/{tablename}/*'))
queries = ['lsqb/sql/q1.sql', 'lsqb/sql/q4.sql']
queries = ['lsqb/sql/q1.sql']
#queries = sorted(glob.glob(f'snap-queries/{tablename}/tree*'))
#queries = ['snap-queries/patents/tree01.sql']

print('running queries: ' + str(queries))
benchmark_all(dbname, 'opt', runs, queries)
#benchmark_all(dbname, 'ref', runs, queries)

running queries: ['lsqb/sql/q1.sql']
person_likes_post
person_studyat_university
person_workat_company
person_knows_person
message
company
university
continent
country
city
tag
tagclass
forum
comment
post
person
comment_hastag_tag
post_hastag_tag
forum_hasmember_person
forum_hastag_tag
person_hasinterest_tag
person_likes_comment
comment_replyof_message
message_hascreator_person
message_hastag_tag
message_islocatedin_country
person_likes_message
benchmark-results-lsqb/results-opt.csv
+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.yannaka...| true|
+--------------------+-----+

running query: 
SELECT count(*)

FROM Country

JOIN City

  ON City.isPartOf_CountryId = Country.CountryId

JOIN Person

  ON Person.isLocatedIn_CityId = City.CityId

JOIN Forum_hasMember_Person

  ON Forum_hasMember_Person.PersonId = Person.PersonId

JOIN Forum

  ON Forum.ForumId = Forum_hasMember_Person.ForumId

JOIN Post

  ON Post.Forum_containerOfId = Forum.

23/11/16 22:46:24 WARN RewriteJoinsAsSemijoins: applying yannakakis rewriting to join: Aggregate [toprettystring(count(1), Some(GMT)) AS toprettystring(count(1))#1112]
+- Project
   +- Join Inner, (hasType_TagClassId#1012L = TagClassId#1015L)
      :- Project [hastype_tagclassid#1012L]
      :  +- Join Inner, (TagId#1011L = TagId#1046L)
      :     :- Project [tagid#1046L]
      :     :  +- Join Inner, (CommentId#1045L = CommentId#1023L)
      :     :     :- Project [commentid#1023L]
      :     :     :  +- Join Inner, (replyOf_PostId#1026L = PostId#1033L)
      :     :     :     :- Project [postid#1033L]
      :     :     :     :  +- Join Inner, (Forum_containerOfId#1035L = ForumId#1019L)
      :     :     :     :     :- Project [forumid#1019L]
      :     :     :     :     :  +- Join Inner, (ForumId#1019L = ForumId#1053L)
      :     :     :     :     :     :- Project [forumid#1053L]
      :     :     :     :     :     :  +- Join Inner, (PersonId#1054L = PersonId#1041L)
      :     :

+-----------+
|   count(1)|
+-----------+
|38292918372|
+-----------+

query id: 123
   Unnamed: 0.1  Unnamed: 0   query     runtime  peak_memory mode run
0           0.0         0.0  q1.sql  371.592946   214.022754  opt   5
1           1.0         NaN  q1.sql  311.109919   196.534317  opt   5
2           NaN         NaN  q1.sql  254.264482   212.651438  opt  05
cancelling jobs with id P29J6DN2FGUGG64O
None
cancelled job
cancelling jobs with id ONHXTBIEDDJQO6PC
None
cancelled job


## TPC-H Benchmark

In [8]:
#### benchmark configuration
group_in_leaves = False
dbname = 'tpch'
#mode = 'opt'
runs = ['01', '02', '03']
#runs = ['04']
#runs = ['01']
####

#queries = sorted(glob.glob(f'snap-queries/{tablename}/*'))
queries = ['tpch-kit/dbgen/queries/postgres/2.sql',
           'tpch-kit/dbgen/queries/postgres/11.sql', 
           'tpch-kit/dbgen/queries/postgres/11-hint.sql']
queries = ['tpch-queries/median-1.sql', 'tpch-queries/median-1-hint.sql']
queries = ['tpch-queries/2-subq.sql'] #, 'tpch-queries/2-subq-hint.sql']

print('running queries: ' + str(queries))
benchmark_all(dbname, 'opt', runs, queries)
benchmark_all(dbname, 'ref', runs, queries)

running queries: ['tpch-queries/2-subq.sql']


23/11/17 14:45:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


part
supplier
partsupp
customer
orders
lineitem
nation
region
benchmark-results-tpch/results-opt.csv
+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.yannaka...| true|
+--------------------+-----+

running query: 
		select

			min(ps_supplycost)

		from

			partsupp,

			supplier,

			nation,

			region

		where

			s_suppkey = ps_suppkey

			and s_nationkey = n_nationkey

			and n_regionkey = r_regionkey

			and r_name = 'EUROPE'


23/11/17 14:45:53 WARN RewriteJoinsAsSemijoins: applying yannakakis rewriting to join: Aggregate [toprettystring(min(ps_supplycost#64), Some(GMT)) AS toprettystring(min(ps_supplycost))#185]
+- Project [ps_supplycost#64]
   +- Join Inner, (n_regionkey#148L = cast(r_regionkey#155 as bigint))
      :- Project [ps_supplycost#64, n_regionkey#148L]
      :  +- Join Inner, (s_nationkey#48L = cast(n_nationkey#146 as bigint))
      :     :- Project [ps_supplycost#64, s_nationkey#48L]
      :     :  +- Join Inner, (cast(s_suppkey#45 as bigint) = ps_suppkey#62L)
      :     :     :- Project [ps_suppkey#62L, ps_supplycost#64]
      :     :     :  +- Filter isnotnull(ps_suppkey#62L)
      :     :     :     +- Relation [ps_partkey#61L,ps_suppkey#62L,ps_availqty#63,ps_supplycost#64,ps_comment#65] JDBCRelation(partsupp) [numPartitions=1]
      :     :     +- Project [s_suppkey#45, s_nationkey#48L]
      :     :        +- Filter (isnotnull(s_suppkey#45) AND isnotnull(s_nationkey#48L))
      :     :    

+--------------------+
|  min(ps_supplycost)|
+--------------------+
|1.000000000000000000|
+--------------------+

query id: 11
    Unnamed: 0.4  Unnamed: 0.3  Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  \
0            0.0           0.0           0.0           0.0         0.0   
1            1.0           1.0           1.0           1.0         0.0   
2            2.0           2.0           2.0           2.0         0.0   
3            3.0           3.0           3.0           3.0         0.0   
4            4.0           4.0           4.0           4.0         0.0   
5            5.0           5.0           5.0           5.0         0.0   
6            6.0           6.0           6.0           6.0         0.0   
7            7.0           7.0           7.0           7.0         0.0   
8            8.0           8.0           8.0           8.0         0.0   
9            9.0           9.0           9.0           0.0         NaN   
10          10.0          10.0          10.0           0.

...

23/11/17 14:49:47 WARN RewriteJoinsAsSemijoins: applying yannakakis rewriting to join: Aggregate [toprettystring(min(ps_supplycost#64), Some(GMT)) AS toprettystring(min(ps_supplycost))#253]
+- Project [ps_supplycost#64]
   +- Join Inner, (n_regionkey#148L = cast(r_regionkey#155 as bigint))
      :- Project [ps_supplycost#64, n_regionkey#148L]
      :  +- Join Inner, (s_nationkey#48L = cast(n_nationkey#146 as bigint))
      :     :- Project [ps_supplycost#64, s_nationkey#48L]
      :     :  +- Join Inner, (cast(s_suppkey#45 as bigint) = ps_suppkey#62L)
      :     :     :- Project [ps_suppkey#62L, ps_supplycost#64]
      :     :     :  +- Filter isnotnull(ps_suppkey#62L)
      :     :     :     +- Relation [ps_partkey#61L,ps_suppkey#62L,ps_availqty#63,ps_supplycost#64,ps_comment#65] JDBCRelation(partsupp) [numPartitions=1]
      :     :     +- Project [s_suppkey#45, s_nationkey#48L]
      :     :        +- Filter (isnotnull(s_suppkey#45) AND isnotnull(s_nationkey#48L))
      :     :    

+--------------------+
|  min(ps_supplycost)|
+--------------------+
|1.000000000000000000|
+--------------------+

query id: 17
    Unnamed: 0.4  Unnamed: 0.3  Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  \
0            0.0           0.0           0.0           0.0         0.0   
1            1.0           1.0           1.0           1.0         0.0   
2            2.0           2.0           2.0           2.0         0.0   
3            3.0           3.0           3.0           3.0         0.0   
4            4.0           4.0           4.0           4.0         0.0   
5            5.0           5.0           5.0           5.0         0.0   
6            6.0           6.0           6.0           6.0         0.0   
7            7.0           7.0           7.0           7.0         0.0   
8            8.0           8.0           8.0           8.0         0.0   
9            9.0           9.0           9.0           0.0         NaN   
10          10.0          10.0          10.0           0.

+--------------------+
|  min(ps_supplycost)|
+--------------------+
|1.000000000000000000|
+--------------------+

query id: 29
    Unnamed: 0.3  Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0              query  \
0            0.0           0.0           0.0         0.0              2.sql   
1            1.0           1.0           1.0         0.0             11.sql   
2            2.0           2.0           2.0         0.0        11-hint.sql   
3            3.0           3.0           3.0         0.0              2.sql   
4            4.0           4.0           4.0         0.0             11.sql   
5            5.0           5.0           5.0         0.0        11-hint.sql   
6            6.0           6.0           6.0         0.0              2.sql   
7            7.0           7.0           7.0         0.0             11.sql   
8            8.0           8.0           8.0         0.0        11-hint.sql   
9            9.0           9.0           0.0         NaN              2.sql   
10

+--------------------+
|  min(ps_supplycost)|
+--------------------+
|1.000000000000000000|
+--------------------+

query id: 32
    Unnamed: 0.3  Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0              query  \
0            0.0           0.0           0.0         0.0              2.sql   
1            1.0           1.0           1.0         0.0             11.sql   
2            2.0           2.0           2.0         0.0        11-hint.sql   
3            3.0           3.0           3.0         0.0              2.sql   
4            4.0           4.0           4.0         0.0             11.sql   
5            5.0           5.0           5.0         0.0        11-hint.sql   
6            6.0           6.0           6.0         0.0              2.sql   
7            7.0           7.0           7.0         0.0             11.sql   
8            8.0           8.0           8.0         0.0        11-hint.sql   
9            9.0           9.0           0.0         NaN              2.sql   
10

+--------------------+
|  min(ps_supplycost)|
+--------------------+
|1.000000000000000000|
+--------------------+

query id: 35
    Unnamed: 0.3  Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0              query  \
0            0.0           0.0           0.0         0.0              2.sql   
1            1.0           1.0           1.0         0.0             11.sql   
2            2.0           2.0           2.0         0.0        11-hint.sql   
3            3.0           3.0           3.0         0.0              2.sql   
4            4.0           4.0           4.0         0.0             11.sql   
5            5.0           5.0           5.0         0.0        11-hint.sql   
6            6.0           6.0           6.0         0.0              2.sql   
7            7.0           7.0           7.0         0.0             11.sql   
8            8.0           8.0           8.0         0.0        11-hint.sql   
9            9.0           9.0           0.0         NaN              2.sql   
10

## JOB (IMDB) Benchmark

In [ ]:
#### benchmark configuration
group_in_leaves = False
dbname = 'imdb'
#mode = 'opt'
runs = ['01', '02']
#runs = ['04']
#runs = ['01']
####

queries = ['job/2a.sql', 'job/2b.sql', 'job/2c.sql', 'job/2d.sql',
           'job/3a.sql', 'job/3b.sql', 'job/3c.sql',
           'job/5a.sql', 'job/5b.sql', 'job/5c.sql',
           'job/17a.sql', 'job/17b.sql', 'job/17c.sql', 'job/17d.sql', 'job/17e.sql', 'job/17f.sql',
           'job/20a.sql', 'job/20b.sql', 'job/20c.sql',
          ]

print('running queries: ' + str(queries))
benchmark_all(dbname, 'opt', runs, queries)
benchmark_all(dbname, 'ref', runs, queries)

running queries: ['job/2a.sql', 'job/2b.sql', 'job/2c.sql', 'job/2d.sql', 'job/3a.sql', 'job/3b.sql', 'job/3c.sql', 'job/5a.sql', 'job/5b.sql', 'job/5c.sql', 'job/17a.sql', 'job/17b.sql', 'job/17c.sql', 'job/17d.sql', 'job/17e.sql', 'job/17f.sql', 'job/20a.sql', 'job/20b.sql', 'job/20c.sql']


23/11/17 21:54:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


name
char_name
company_name
kind_type
title
company_type
aka_name
aka_title
role_type
cast_info
comp_cast_type
complete_cast
info_type
link_type
keyword
movie_keyword
movie_link
movie_info
movie_companies
person_info
+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.yannaka...| true|
+--------------------+-----+

running query: 
SELECT MIN(t.title) AS movie_title

FROM company_name AS cn,

     keyword AS k,

     movie_companies AS mc,

     movie_keyword AS mk,

     title AS t

WHERE cn.country_code ='[de]'

  AND k.keyword ='character-name-in-title'

  AND cn.id = mc.company_id

  AND mc.movie_id = t.id

  AND t.id = mk.movie_id

  AND mk.keyword_id = k.id

  AND mc.movie_id = mk.movie_id;





23/11/17 21:54:07 WARN RewriteJoinsAsSemijoins: applying yannakakis rewriting to join: Aggregate [toprettystring(min(title#75), Some(GMT)) AS toprettystring(movie_title)#253]
+- Project [title#75]
   +- Join Inner, ((movie_id#211 = id#74) AND (id#74 = movie_id#187))
      :- Project [movie_id#211, movie_id#187]
      :  +- Join Inner, (keyword_id#188 = id#180)
      :     :- Join Inner, (movie_id#211 = movie_id#187)
      :     :  :- Project [movie_id#211]
      :     :  :  +- Join Inner, (id#56 = company_id#212)
      :     :  :     :- Project [id#56]
      :     :  :     :  +- Filter ((isnotnull(country_code#58) AND (country_code#58 = [de])) AND isnotnull(id#56))
      :     :  :     :     +- Relation [id#56,name#57,country_code#58,imdb_id#59,name_pcode_nf#60,name_pcode_sf#61,md5sum#62] JDBCRelation(company_name) [numPartitions=1]
      :     :  :     +- Project [movie_id#211, company_id#212]
      :     :  :        +- Filter (isnotnull(company_id#212) AND isnotnull(movie_id#211))
  

+--------------------+
|         movie_title|
+--------------------+
|'Crocodile' Dunde...|
+--------------------+

query id: 23


...

+--------------------+
|         movie_title|
+--------------------+
|300: Rise of an E...|
+--------------------+

query id: 230
      query     runtime  peak_memory mode run
0    2a.sql    5.791678    16.389018  ref  01
1    2b.sql    5.527152    13.010227  ref  01
2    2c.sql    4.455633    11.681939  ref  01
3    2d.sql    8.097607    21.644059  ref  01
4    3a.sql    5.298154    22.215893  ref  01
5    3b.sql    5.543727    13.042463  ref  01
6    3c.sql    5.429142     9.733444  ref  01
7    5a.sql    0.379339    10.035139  ref  01
8    5b.sql    0.438972     8.459837  ref  01
9    5c.sql    2.806309     6.940926  ref  01
10  17a.sql   72.302747   208.803557  ref  01
11  17b.sql  139.620638   404.680286  ref  01
12  17c.sql  133.036928   414.358938  ref  01
13  17d.sql  139.114643   418.326929  ref  01
14  17e.sql   78.391719   426.911244  ref  01
15  17f.sql  143.456903   417.334526  ref  01
16  20a.sql   28.578576   426.471670  ref  01
17  20b.sql   22.720177   229.806023  ref 

+-----------+
|movie_title|
+-----------+
| & Teller 2|
+-----------+

query id: 233
      query     runtime  peak_memory mode run
0    2a.sql    5.791678    16.389018  ref  01
1    2b.sql    5.527152    13.010227  ref  01
2    2c.sql    4.455633    11.681939  ref  01
3    2d.sql    8.097607    21.644059  ref  01
4    3a.sql    5.298154    22.215893  ref  01
5    3b.sql    5.543727    13.042463  ref  01
6    3c.sql    5.429142     9.733444  ref  01
7    5a.sql    0.379339    10.035139  ref  01
8    5b.sql    0.438972     8.459837  ref  01
9    5c.sql    2.806309     6.940926  ref  01
10  17a.sql   72.302747   208.803557  ref  01
11  17b.sql  139.620638   404.680286  ref  01
12  17c.sql  133.036928   414.358938  ref  01
13  17d.sql  139.114643   418.326929  ref  01
14  17e.sql   78.391719   426.911244  ref  01
15  17f.sql  143.456903   417.334526  ref  01
16  20a.sql   28.578576   426.471670  ref  01
17  20b.sql   22.720177   229.806023  ref  01
18  20c.sql   25.693910    49.548177  ref

running query: 
SELECT MIN(t.title) AS american_vhs_movie

FROM company_type AS ct,

     info_type AS it,

     movie_companies AS mc,

     movie_info AS mi,

     title AS t

WHERE ct.kind = 'production companies'

  AND mc.note LIKE '%(VHS)%'

  AND mc.note LIKE '%(USA)%'

  AND mc.note LIKE '%(1994)%'

  AND mi.info IN ('USA',

                  'America')

  AND t.production_year > 2010

  AND t.id = mi.movie_id

  AND t.id = mc.movie_id

  AND mc.movie_id = mi.movie_id

  AND ct.id = mc.company_type_id

  AND it.id = mi.info_type_id;



+------------------+
|american_vhs_movie|
+------------------+
|              NULL|
+------------------+

query id: 239
      query     runtime  peak_memory mode run
0    2a.sql    5.791678    16.389018  ref  01
1    2b.sql    5.527152    13.010227  ref  01
2    2c.sql    4.455633    11.681939  ref  01
3    2d.sql    8.097607    21.644059  ref  01
4    3a.sql    5.298154    22.215893  ref  01
5    3b.sql    5.543727    13.042463  ref  01
6    3c.

+--------------+
|american_movie|
+--------------+
|    11,830,420|
+--------------+

query id: 242
      query     runtime  peak_memory mode run
0    2a.sql    5.791678    16.389018  ref  01
1    2b.sql    5.527152    13.010227  ref  01
2    2c.sql    4.455633    11.681939  ref  01
3    2d.sql    8.097607    21.644059  ref  01
4    3a.sql    5.298154    22.215893  ref  01
5    3b.sql    5.543727    13.042463  ref  01
6    3c.sql    5.429142     9.733444  ref  01
7    5a.sql    0.379339    10.035139  ref  01
8    5b.sql    0.438972     8.459837  ref  01
9    5c.sql    2.806309     6.940926  ref  01
10  17a.sql   72.302747   208.803557  ref  01
11  17b.sql  139.620638   404.680286  ref  01
12  17c.sql  133.036928   414.358938  ref  01
13  17d.sql  139.114643   418.326929  ref  01
14  17e.sql   78.391719   426.911244  ref  01
15  17f.sql  143.456903   417.334526  ref  01
16  20a.sql   28.578576   426.471670  ref  01
17  20b.sql   22.720177   229.806023  ref  01
18  20c.sql   25.693910   

+----------------------------------+---------+
|member_in_charnamed_american_movie|       a1|
+----------------------------------+---------+
|                         B., Billy|B., Billy|
+----------------------------------+---------+

query id: 245
      query     runtime  peak_memory mode run
0    2a.sql    5.791678    16.389018  ref  01
1    2b.sql    5.527152    13.010227  ref  01
2    2c.sql    4.455633    11.681939  ref  01
3    2d.sql    8.097607    21.644059  ref  01
4    3a.sql    5.298154    22.215893  ref  01
5    3b.sql    5.543727    13.042463  ref  01
6    3c.sql    5.429142     9.733444  ref  01
7    5a.sql    0.379339    10.035139  ref  01
8    5b.sql    0.438972     8.459837  ref  01
9    5c.sql    2.806309     6.940926  ref  01
10  17a.sql   72.302747   208.803557  ref  01
11  17b.sql  139.620638   404.680286  ref  01
12  17c.sql  133.036928   414.358938  ref  01
13  17d.sql  139.114643   418.326929  ref  01
14  17e.sql   78.391719   426.911244  ref  01
15  17f.sql  1